# Layer Skip Sanity Check

验证 `generate_with_layer_skip` 在 `layer_skip_threshold=1.0` 时退化成普通 `generate_with_dual_cache` 版本。

**验证逻辑：**
- 当 threshold=1.0 时，cos_sim 不可能 > 1，所以不会跳过任何层
- 因此结果应该与普通 dual_cache 完全一致

## 1. 环境设置

In [1]:
import os
import torch
import gc

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.chdir('llada')

torch.cuda.empty_cache()
gc.collect()

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: NVIDIA GeForce RTX 3090 Ti


## 2. 加载模型

In [2]:
from transformers import AutoTokenizer
from model.modeling_llada import LLaDAModelLM

device = 'cuda'
model_name = 'GSAI-ML/LLaDA-8B-Instruct'

print(f"Loading model: {model_name}")
model = LLaDAModelLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).to(device).eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print("✅ Model loaded!")

/home/pianng/miniconda3/envs/dllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading model: GSAI-ML/LLaDA-8B-Instruct


Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00,  8.86it/s]


✅ Model loaded!


In [3]:
from generate import (
    generate_with_dual_cache,
    generate_with_layer_skip,
)

print("✅ Functions imported!")

✅ Functions imported!


## 3. Sanity Check: threshold=1.0 应该与 dual_cache 完全一致

In [4]:
def prepare_input(prompt_text, tokenizer, device):
    m = [{"role": "user", "content": prompt_text}]
    prompt = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
    input_ids = tokenizer(prompt)['input_ids']
    input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
    return input_ids

In [5]:
# 测试用的 prompts
test_prompts = [
    "What is 2 + 2?",
    "Who is Newton?",
    "Lily can run 12 km/h for 4 hours. After that, she runs 6 km/h. How many km can she run in 8 hours?",
    "Explain the concept of machine learning in one sentence.",
]

# 生成参数
gen_params = {
    'steps': 32,
    'gen_length': 64,
    'block_length': 32,
    'temperature': 0.,
    'remasking': 'low_confidence',
    'threshold': 0.9,
}

print("=" * 70)
print("Sanity Check: layer_skip_threshold=1.0 vs dual_cache")
print("=" * 70)

all_passed = True

for i, prompt_text in enumerate(test_prompts):
    print(f"\n[Test {i+1}/{len(test_prompts)}] {prompt_text[:40]}...")
    
    input_ids = prepare_input(prompt_text, tokenizer, device)
    prompt_len = input_ids.shape[1]
    
    # 1. 原版 dual_cache
    x_orig, nfe_orig = generate_with_dual_cache(model, input_ids, **gen_params)
    answer_orig = tokenizer.decode(x_orig[0, prompt_len:], skip_special_tokens=True)
    
    # 2. layer_skip with threshold=1.0 (should be identical)
    x_skip, nfe_skip, stats = generate_with_layer_skip(
        model, input_ids, layer_skip_threshold=1.0, **gen_params
    )
    answer_skip = tokenizer.decode(x_skip[0, prompt_len:], skip_special_tokens=True)
    
    # 验证
    tokens_match = torch.equal(x_orig, x_skip)
    nfe_match = nfe_orig == nfe_skip
    
    passed = tokens_match and nfe_match
    all_passed = all_passed and passed
    
    status = "✅ PASSED" if passed else "❌ FAILED"
    print(f"   {status}")
    print(f"   Tokens match: {tokens_match}")
    print(f"   NFE: orig={nfe_orig}, skip={nfe_skip}")
    print(f"   Skip stats: computed={stats['total_layers_computed']}, skipped={stats['total_layers_skipped']}")
    
    if not tokens_match:
        print(f"   ⚠️  Answer orig: {answer_orig[:50]}...")
        print(f"   ⚠️  Answer skip: {answer_skip[:50]}...")

print("\n" + "=" * 70)
if all_passed:
    print("🎉 Sanity Check PASSED! threshold=1.0 correctly degrades to dual_cache.")
else:
    print("❌ Sanity Check FAILED! There's a bug in the implementation.")

Sanity Check: layer_skip_threshold=1.0 vs dual_cache

[Test 1/4] What is 2 + 2?...
   ✅ PASSED
   Tokens match: True
   NFE: orig=5, skip=5
   Skip stats: computed=160, skipped=0

[Test 2/4] Who is Newton?...
   ✅ PASSED
   Tokens match: True
   NFE: orig=17, skip=17
   Skip stats: computed=544, skipped=0

[Test 3/4] Lily can run 12 km/h for 4 hours. After ...
   ✅ PASSED
   Tokens match: True
   NFE: orig=32, skip=32
   Skip stats: computed=1024, skipped=0

[Test 4/4] Explain the concept of machine learning ...
   ✅ PASSED
   Tokens match: True
   NFE: orig=23, skip=23
   Skip stats: computed=736, skipped=0

🎉 Sanity Check PASSED! threshold=1.0 correctly degrades to dual_cache.


## 4. 测试不同 threshold 下的跳过情况

In [8]:
# 测试不同 threshold
test_prompt = "Lily can run 12 km/h for 4 hours. After that, she runs 6 km/h. How many km can she run in 8 hours?"
input_ids = prepare_input(test_prompt, tokenizer, device)
prompt_len = input_ids.shape[1]

# 根据目标 skip_rate 设定的 threshold
thresholds = [1.000, 0.993, 0.989, 0.985, 0.981, 0.978, 0.974, 0.970, 0.966, 0.962, 0.958]

print("\n" + "=" * 70)
print("Testing different layer_skip_thresholds")
print("=" * 70)

# Baseline
x_baseline, nfe_baseline = generate_with_dual_cache(model, input_ids, **gen_params)
answer_baseline = tokenizer.decode(x_baseline[0, prompt_len:], skip_special_tokens=True)
print(f"\nBaseline (dual_cache): NFE={nfe_baseline}")
print(f"Answer: {answer_baseline}")

results = []
for thresh in thresholds:
    x, nfe, stats = generate_with_layer_skip(
        model, input_ids, layer_skip_threshold=thresh, **gen_params
    )
    answer = tokenizer.decode(x[0, prompt_len:], skip_special_tokens=True)
    match = torch.equal(x, x_baseline)
    
    results.append({
        'threshold': thresh,
        'nfe': nfe,
        'computed': stats['total_layers_computed'],
        'skipped': stats['total_layers_skipped'],
        'skip_rate': stats['skip_rate'],
        'match_baseline': match,
        'answer': answer,
    })
    
    print(f"\nThreshold={thresh:.3f}:")
    print(f"   Computed: {stats['total_layers_computed']}, Skipped: {stats['total_layers_skipped']}")
    print(f"   Skip rate: {stats['skip_rate']*100:.1f}%")
    print(f"   Match baseline: {'✅' if match else '❌'}")
    if not match:
        print(f"   Answer: {answer[:60]}...")


Testing different layer_skip_thresholds

Baseline (dual_cache): NFE=32
Answer: In the first 4 hours, Lily runs 12 km/h 4 = 48 km.
 the 6 km/h 4 = 48 km + 24 km = 72 km.
: 72

Threshold=1.000:
   Computed: 1024, Skipped: 0
   Skip rate: 0.0%
   Match baseline: ✅

Threshold=0.993:
   Computed: 993, Skipped: 31
   Skip rate: 3.0%
   Match baseline: ✅

Threshold=0.989:
   Computed: 938, Skipped: 86
   Skip rate: 8.4%
   Match baseline: ❌
   Answer: In the first 4 hours, runs 12 km/h 4 = 48 km.
In the 4 hours...

Threshold=0.985:
   Computed: 899, Skipped: 125
   Skip rate: 12.2%
   Match baseline: ❌
   Answer: In the first 4 hours, she runs 12 km/h 4 = 48 km.
 the she r...

Threshold=0.981:
   Computed: 851, Skipped: 173
   Skip rate: 16.9%
   Match baseline: ❌
   Answer: In the first 4 hours, Lily runs 12 km/h 4 = 48 km.
 the she ...

Threshold=0.978:
   Computed: 682, Skipped: 150
   Skip rate: 18.0%
   Match baseline: ❌
   Answer: In the first 4 hours, 4 hours x 12 km/h = 48 km.
In the

In [7]:
import pandas as pd

df = pd.DataFrame(results)
df['skip_rate'] = df['skip_rate'].apply(lambda x: f"{x*100:.1f}%")
df['match_baseline'] = df['match_baseline'].apply(lambda x: '✅' if x else '❌')
df['answer'] = df['answer'].apply(lambda x: x[:40] + '...' if len(x) > 40 else x)

print("\nSummary:")
display(df[['threshold', 'computed', 'skipped', 'skip_rate', 'match_baseline']])


Summary:


,threshold,computed,skipped,skip_rate,match_baseline
0,1.000,1024,0,0.0%,✅
1,0.993,993,31,3.0%,✅
2,0.989,938,86,8.4%,❌
3,0.985,899,125,12.2%,❌
4,0.981,851,173,16.9%,❌
5,0.978,682,150,18.0%,❌
6,0.974,828,196,19.1%,❌
7,0.970,817,207,20.2%,❌
8,0.966,440,104,19.1%,❌
9,0.962,431,113,20.8%,❌


## 5. 结论

1. **Sanity Check**: `layer_skip_threshold=1.0` 时，结果与 `generate_with_dual_cache` 完全一致
2. **Layer Skip 效果**: 随着 threshold 降低，更多层被跳过，但可能影响生成质量